In [12]:
import csv
import pandas as pd
import os
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM,Dense,Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam

In [13]:
data = pd.read_csv('Netflix_alltime.csv')
data = data.iloc[::-1]

In [14]:
time_steps=10

In [15]:
# Preprocessing data
data = data.drop(columns='Date')
data['Close/Last'] = data['Close/Last'].str.replace('$','')
data['Open'] = data['Open'].str.replace('$','')
data['High'] = data['High'].str.replace('$','')
data['Low'] = data['Low'].str.replace('$','')


# Splitting data into input features (X) and target variable (y)
X_ts = []
y_ts = []
for i in range(len(data) - time_steps):
    X_ts.append(data.iloc[i:i+time_steps].values)
    y_ts.append(data.iloc[i+time_steps].values)

X_ts = np.array(X_ts).astype(np.float32)
y_ts = np.array(y_ts).astype(np.float32)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_ts, y_ts, test_size=0.2, random_state=42)

# Normalize input features (X)
scaler_X = MinMaxScaler()
X_train_normalized = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape) ## convert it into a single row matrix and then normalise then reshape it back
X_test_normalized = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Normalize target variable (y)
scaler_y = MinMaxScaler()
y_train_normalized = scaler_y.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
y_test_normalized = scaler_y.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

# Define and compile the model
model = Sequential()

model.add(LSTM(units=128, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))

# Add the ReLU activation function
model.add(Activation('relu'))

# Add the Dense layers
model.add(Dense(units=128))
model.add(Dense(units=5))  # Output layer with 5 units

# Compile the model with a custom learning rate
custom_learning_rate = 0.001  # You can adjust this value
optimizer = Adam(learning_rate=custom_learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
model.fit(X_train_normalized, y_train_normalized, epochs=200, batch_size=32, validation_data=(X_test_normalized, y_test_normalized))

# Evaluate the model
loss = model.evaluate(X_test_normalized, y_test_normalized)
print('Test Loss:', loss)

# Make predictions for the next step
X_final_test = X_ts[-1].reshape(1, X_ts.shape[1], X_ts.shape[2])  # Using the last data point as input
X_final_test_normalized = scaler_X.transform(X_final_test.reshape(-1, X_final_test.shape[-1])).reshape(X_final_test.shape)
predicted_normalized = model.predict(X_final_test_normalized)

predicted = scaler_y.inverse_transform(predicted_normalized.reshape(-1, predicted_normalized.shape[-1])).reshape(predicted_normalized.shape).astype(int)

print('Predicted:', predicted)


Epoch 1/200
63/63 [==============================] - 2s 10ms/step - loss: 0.0150 - val_loss: 0.0022
Epoch 2/200
63/63 [==============================] - 0s 5ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 3/200
63/63 [==============================] - 0s 4ms/step - loss: 0.0011 - val_loss: 0.0018
Epoch 4/200
63/63 [==============================] - 0s 4ms/step - loss: 0.0010 - val_loss: 0.0018
Epoch 5/200
63/63 [==============================] - 0s 4ms/step - loss: 9.9640e-04 - val_loss: 0.0017
Epoch 6/200
63/63 [==============================] - 0s 5ms/step - loss: 9.9448e-04 - val_loss: 0.0016
Epoch 7/200
63/63 [==============================] - 0s 5ms/step - loss: 8.7241e-04 - val_loss: 0.0016
Epoch 8/200
63/63 [==============================] - 0s 5ms/step - loss: 8.6320e-04 - val_loss: 0.0016
Epoch 9/200
63/63 [==============================] - 0s 5ms/step - loss: 8.7856e-04 - val_loss: 0.0015
Epoch 10/200
63/63 [==============================] - 0s 5ms/step - loss: 8.3520e-04 - v